Mounted at /content/gdrive


In [2]:
#Connect to data folder of NerLTR-DTA
root_path = '/data/'
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


In [3]:
import numpy as np
import pandas as pd
import linecache
import math
from collections import Counter
import pickle

#Paths KIBA dataset

In [4]:
path_KIBA_affinity = 'KIBA/kiba_binding_affinity_v2.txt' #Affinity Matrix AffiDT from Kiba Dataset
path_KIBA_SSAF_drugs = 'KIBA/SAF/SSAF_drugs' #SSAF feature of drug
path_KIBA_SSAF_targets = 'KIBA/SAF/SSAF_targets' #SSAF feature of target
path_KIBA_ASAF_drugs = 'KIBA/SAF/ASAF_drugs' #ASAF feature of drug
path_KIBA_ASAF_targets = 'KIBA/SAF/ASAF_targets' #ASAF feature of target
path_KIBA_GAAF_drugs = 'KIBA/AAF/GAAF_drugs' #GAAF feature of drugs
path_KIBA_GAAF_targets = 'KIBA/AAF/GAAF_targets' #GAAF feature of target
path_KIBA_AAAF_drugs = 'KIBA/AAF/AAAF_drugs' #AAAF feature of drugs
path_KIBA_AAAF_targets = 'KIBA/AAF/AAAF_targets' #AAAF feature of target

In [5]:
path_KIBA_fold_train = 'KIBA/fold/train_fold_setting1.txt'
path_KIBA_fold_test = 'KIBA/fold/test_fold_setting1.txt'

#Generate test and train files for RankLib

In [8]:
from ast import literal_eval

##KIBA

In [9]:
with open(path_KIBA_fold_test) as f:
    test_KIBA = literal_eval(f.read())
print(f'Test {len(test_KIBA)}')

Test 19709


In [10]:
#check for highes index in 
np.max(test_KIBA)

118241

In [11]:
with open(path_KIBA_fold_train) as f:
    train_KIBA = literal_eval(f.read())
print(f'Fold1 {len(train_KIBA[0])}')
print(f'Fold2 {len(train_KIBA[1])}')
print(f'Fold3 {len(train_KIBA[2])}')
print(f'Fold4 {len(train_KIBA[3])}')
print(f'Fold5 {len(train_KIBA[4])}')
#arr[0] # check ids if fold with data fold file

Fold1 19709
Fold2 19709
Fold3 19709
Fold4 19709
Fold5 19709


In [12]:
#check for highes index in folds --> must not be higher than 118253
np.max(train_KIBA)

118253

In [13]:
#load Affinity Matrix
aff_KIBA = np.loadtxt(path_KIBA_affinity)

In [14]:
# counter to check how many elements in dataset are not nan e.g. KIBA 118254
counter = 0
#dataframe for Drug ID, Target ID plus Affinity and general ID 
kiba_df = pd.DataFrame(columns = ['drug_ID', 'target_ID', 'affinity'])
for i,row in enumerate(aff_KIBA):
  for j,el in enumerate(row):
    if el > -1:
      counter +=1
      data = {
        'drug_ID':i,
        'target_ID':j,
        'affinity': el
        }

      kiba_df=kiba_df.append(data, ignore_index=True)
kiba_df.drug_ID = kiba_df.drug_ID.astype(int)
kiba_df.target_ID = kiba_df.target_ID.astype(int)
print(f'Counter: {counter}')

Counter: 118254


In [15]:
#check df
kiba_df

,drug_ID,target_ID,affinity
0,0,0,11.10000
1,0,6,11.10000
2,0,9,11.10000
3,0,25,11.10000
4,0,27,11.10000
...,...,...,...
118249,2110,154,10.49794
118250,2110,155,10.49794
118251,2110,156,10.49794
118252,2110,173,10.49794


In [16]:
kiba_train = kiba_df.iloc[train_KIBA[0]+train_KIBA[1]+train_KIBA[2]+train_KIBA[3]+train_KIBA[4]].copy()
kiba_fold1 = kiba_df.iloc[train_KIBA[0]].copy()
kiba_fold2 = kiba_df.iloc[train_KIBA[1]].copy()
kiba_fold3 = kiba_df.iloc[train_KIBA[2]].copy()
kiba_fold4 = kiba_df.iloc[train_KIBA[3]].copy()
kiba_fold5 = kiba_df.iloc[train_KIBA[4]].copy()
kiba_test = kiba_df.iloc[test_KIBA].copy()

In [17]:
kiba_train

,drug_ID,target_ID,affinity
113703,1776,20,9.798970
51222,492,160,11.400000
98078,1207,204,11.200000
29622,305,98,11.200000
80976,917,46,11.999998
...,...,...,...
88379,1017,96,13.800057
98714,1218,39,11.600000
11884,158,3,11.500000
90741,1080,36,10.600000


In [18]:
kiba_test

,drug_ID,target_ID,affinity
34121,345,83,14.400162
51548,495,104,12.399998
12611,163,106,11.400000
104850,1372,222,10.100000
23744,254,97,12.699998
...,...,...,...
48284,467,159,11.800001
84487,967,141,11.200000
5856,104,176,11.400000
84505,968,4,11.500000


In [19]:
kiba_train.to_pickle("affi/train_aff_KIBA.pkl")
kiba_fold1.to_pickle("affi/train_aff_fold1_KIBA.pkl")
kiba_fold2.to_pickle("affi/train_aff_fold2_KIBA.pkl")
kiba_fold3.to_pickle("affi/train_aff_fold3_KIBA.pkl")
kiba_fold4.to_pickle("affi/train_aff_fold4_KIBA.pkl")
kiba_fold5.to_pickle("affi/train_aff_fold5_KIBA.pkl")
kiba_test.to_pickle("affi/test_aff_KIBA.pkl")

Test 5010


30054

Fold1 5010
Fold2 5009
Fold3 5009
Fold4 5009
Fold5 5009


30050
30046
30053
30055
30051


Counter: 30056


,drug_ID,target_ID,affinity
0,0,0,7.366532
1,0,1,5.000000
2,0,2,5.000000
3,0,3,5.000000
4,0,4,5.000000
...,...,...,...
30051,67,437,5.000000
30052,67,438,5.000000
30053,67,439,5.721246
30054,67,440,5.356547


,drug_ID,target_ID,affinity
1483,3,157,5.000000
18097,40,417,5.000000
11736,26,244,5.000000
16204,36,292,5.000000
2085,4,317,5.000000
...,...,...,...
5060,11,198,5.000000
3531,7,437,5.000000
21824,49,166,5.000000
1017,2,133,6.744727


,drug_ID,target_ID,affinity
9331,21,49,5.000000
10366,23,200,5.000000
13309,30,49,5.000000
9889,22,165,8.823909
26788,60,268,5.000000
...,...,...,...
7533,17,19,6.677781
24146,54,278,5.000000
3676,8,140,5.000000
16432,37,78,5.000000


##Load feature files

###KIBA

In [31]:
#drugs
with open(path_KIBA_SSAF_drugs, "rb") as fp:   # Unpickling SSAF features drugs
    ssaf_drugs_KIBA = pickle.load(fp)
with open(path_KIBA_ASAF_drugs, "rb") as fp:   # Unpickling ASAF features drugs
    asaf_drugs_KIBA = pickle.load(fp)
with open(path_KIBA_GAAF_drugs, "rb") as fp:   # Unpickling GAAF features drugs
    gaaf_drugs_KIBA = pickle.load(fp)
with open(path_KIBA_AAAF_drugs, "rb") as fp:   # Unpickling AAAF features drugs
    aaaf_drugs_KIBA = pickle.load(fp)

#targets
with open(path_KIBA_SSAF_targets, "rb") as fp:   # Unpickling SSAF features targets
    ssaf_targets_KIBA = pickle.load(fp)
with open(path_KIBA_ASAF_targets, "rb") as fp:   # Unpickling ASAF features targets
    asaf_targets_KIBA = pickle.load(fp)
with open(path_KIBA_GAAF_targets, "rb") as fp:   # Unpickling GAAF features targets
    gaaf_targets_KIBA = pickle.load(fp)
with open(path_KIBA_AAAF_targets, "rb") as fp:   # Unpickling AAAF features targets
    aaaf_targets_KIBA = pickle.load(fp)

In [32]:
print(len(ssaf_drugs_KIBA))
print(len(asaf_drugs_KIBA))
print(len(gaaf_drugs_KIBA))
print(len(aaaf_drugs_KIBA))

2111
2111
2111
2111


In [33]:
print(len(ssaf_targets_KIBA))
print(len(asaf_targets_KIBA))
print(len(gaaf_targets_KIBA))
print(len(aaaf_targets_KIBA))

229
229
229
229


In [34]:
ssaf_drugs_KIBA

,mean,median,75q,85q,95q
0,0.381405,0.366,0.4360,0.4675,0.5070
1,0.387980,0.372,0.4430,0.4760,0.5165
2,0.551189,0.567,0.6200,0.6480,0.6870
3,0.572271,0.587,0.6700,0.7140,0.7865
4,0.581582,0.597,0.6560,0.6850,0.7270
...,...,...,...,...,...
2106,0.336646,0.314,0.3810,0.4150,0.5680
2107,0.338769,0.316,0.3840,0.4160,0.5725
2108,0.408101,0.400,0.4550,0.4780,0.5115
2109,0.557937,0.558,0.6155,0.6530,0.7570


In [35]:
asaf_drugs_KIBA

,mean_1,mean_2,number,mode,25q,50q,75q,5_high,5_low
0,0.877293,11.161111,18,11.100000,11.100000,11.100000,11.100000,"[11.1, 11.1, 11.200000128, 12.199999035, 12.99...","[10.1, 10.1, 11.1, 11.1, 11.1]"
1,0.875764,11.141667,18,11.100000,11.100000,11.100000,11.100000,"[11.1, 11.1, 11.200000128, 11.999998417, 12.85...","[10.1, 10.1, 11.1, 11.1, 11.1]"
2,6.513864,12.127439,123,11.200000,11.450000,12.112837,12.380846,"[13.816269724, 13.986582248, 14.113255031, 14....","[11.1, 11.1, 11.1, 11.1, 11.1]"
3,0.918660,11.687402,18,11.100000,11.100000,11.421414,12.157882,"[12.175720714, 12.504466153, 12.515897595, 12....","[11.1, 11.1, 11.1, 11.1, 11.1]"
4,3.405819,12.186447,64,12.100000,12.100000,12.100000,12.100000,"[12.555931956, 12.798970004, 13.020818754, 13....","[11.798970004, 11.798970004, 11.798970004, 11...."
...,...,...,...,...,...,...,...,...,...
2106,0.589416,11.248023,12,10.100000,10.100000,10.946895,12.014542,"[11.953871964, 11.986056648, 12.1, 12.58148606...","[10.1, 10.1, 10.1, 10.1, 10.392429824]"
2107,0.546941,11.386322,11,10.100000,10.340743,11.020819,12.412077,"[11.986056648, 12.242667504, 12.58148606, 12.6...","[10.1, 10.1, 10.1, 10.58148606, 10.777780705]"
2108,0.630685,13.129721,11,12.986057,12.660739,12.986057,13.372578,"[12.986056648, 13.122276395, 13.622878745, 13....","[12.622878745, 12.637602002, 12.652841969, 12...."
2109,0.680002,11.978493,13,12.765546,11.361219,11.633132,12.765546,"[12.765546249, 12.765546249, 12.765546249, 12....","[10.794648631, 10.798970004, 11.101304842, 11...."


In [36]:
gaaf_drugs_KIBA

,min,max,upper_q,median,lower_q,mean,mode
0,0.000000,166.0,12.599997,11.300000,11.1,14.600535,11.1
1,0.000000,166.0,12.599997,11.300000,11.1,14.590844,11.1
2,0.000000,166.0,13.000000,11.588833,11.1,15.491607,11.1
3,0.498253,166.0,12.899998,11.608638,11.1,14.989021,11.1
4,0.475089,166.0,12.899998,11.538074,11.1,15.375821,11.1
...,...,...,...,...,...,...,...
2106,0.000000,166.0,12.463643,11.300000,10.8,13.825336,11.1
2107,0.000000,166.0,12.489340,11.300000,10.8,13.826116,11.1
2108,0.000000,166.0,12.599997,11.400000,11.1,14.887384,11.1
2109,0.470693,166.0,12.899998,11.500000,11.1,14.240340,11.1


In [37]:
#aaaf_drugs_KIBA

68
68
68
68


442
442
442
442


#Write Train and Test File

In [41]:
def write_file(outpath:str, dataframe, ssaf_drugs, asaf_drugs, gaaf_drugs, aaaf_drugs, ssaf_targets, asaf_targets, gaaf_targets, aaaf_targets):
  with open(outpath, 'w') as f:
      for index, row in dataframe.iterrows():
        #write affinity
        f.write(str(row['affinity']))
        #write qid
        f.write(' qid:')
        f.write(str(int(row['drug_ID'])))
        f.write(' ')
        #write features
        feature_number = 1  # for Ranklib thte feature number has to be infront of each feature
        #DRUGS
        #ssaf
        ssaf_dict = {'mean', 'median', '75q', '85q', '95q'}
        for entry in ssaf_dict:
          f.write(str(feature_number)+':')
          f.write(str(ssaf_drugs[entry][row['drug_ID']]))
          f.write(' ')
          feature_number +=1
        #asaf
        asaf_dict ={'mean_1','mean_2','number','mode','25q','50q','75q'}
        for entry in asaf_dict:
          f.write(str(feature_number)+':')
          f.write(str(asaf_drugs[entry][row['drug_ID']]))
          f.write(' ')
          feature_number +=1

        for el in asaf_drugs['5_high'][row['drug_ID']]:
          f.write(str(feature_number)+':')
          f.write(str(el))
          f.write(' ')
          feature_number +=1
        for el in asaf_drugs['5_low'][row['drug_ID']]:
          f.write(str(feature_number)+':')
          f.write(str(el))
          f.write(' ')
          feature_number +=1
        #gaaf
        gaaf_dict ={'min','max','upper_q','median','lower_q','mean','mode'}
        for entry in gaaf_dict:
          f.write(str(feature_number)+':')
          f.write(str(gaaf_drugs[entry][row['drug_ID']]))
          f.write(' ')
          feature_number +=1
        #aaaf
        for el in aaaf_drugs[int(row['drug_ID'])]:
          f.write(str(feature_number)+':')
          f.write(str(el))
          f.write(' ')
          feature_number +=1

        #Targets
        #ssaf
        ssaf_dict = {'mean', 'median', '75q', '85q', '95q'}
        for entry in ssaf_dict:
          f.write(str(feature_number)+':')
          f.write(str(ssaf_targets[entry][row['target_ID']]))
          f.write(' ')
          feature_number +=1
        #asaf
        asaf_dict ={'mean_1','mean_2','number','mode','25q','50q','75q'}
        for entry in asaf_dict:
          f.write(str(feature_number)+':')
          f.write(str(asaf_targets[entry][row['target_ID']]))
          f.write(' ')
          feature_number +=1

        for el in asaf_targets['5_high'][row['target_ID']]:
          f.write(str(feature_number)+':')
          f.write(str(el))
          f.write(' ')
          feature_number +=1
        for el in asaf_targets['5_low'][row['target_ID']]:
          f.write(str(feature_number)+':')
          f.write(str(el))
          f.write(' ')
          feature_number +=1
        #gaaf
        gaaf_dict ={'min','max','upper_q','median','lower_q','mean','mode'}
        for entry in gaaf_dict:
          f.write(str(feature_number)+':')
          f.write(str(gaaf_targets[entry][row['target_ID']]))
          f.write(' ')
          feature_number +=1
        #aaaf
        for el in aaaf_targets[int(row['target_ID'])]:
          f.write(str(feature_number)+':')
          f.write(str(el))
          f.write(' ')
          feature_number +=1
        f.write('\n')
      

In [42]:
write_file('train_KIBA_Ner.txt', kiba_train, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

In [44]:
write_file('features/combined_RankLib/train_fold1_KIBA.txt', kiba_fold1, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file('features/combined_RankLib/train_fold2_KIBA.txt', kiba_fold2, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file('features/combined_RankLib/train_fold3_KIBA.txt', kiba_fold3, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file('features/combined_RankLib/train_fold4_KIBA.txt', kiba_fold4, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file('features/combined_RankLib/train_fold5_KIBA.txt', kiba_fold5, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

In [45]:
write_file('test_KIBA_Ner.txt', kiba_test, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

#Only features without RankLib format

##Write file with Drugs and Targets combined

In [47]:
def write_file_RAW(outpath:str, dataframe, ssaf_drugs, asaf_drugs, gaaf_drugs, aaaf_drugs, ssaf_targets, asaf_targets, gaaf_targets, aaaf_targets):
  with open(outpath, 'w') as f:
      for index, row in dataframe.iterrows():
        #DRUGS
        #ssaf
        ssaf_dict = {'mean', 'median', '75q', '85q', '95q'}
        for entry in ssaf_dict:
          f.write(str(ssaf_drugs[entry][row['drug_ID']]))
          f.write(' ')
        #asaf
        asaf_dict ={'mean_1','mean_2','number','mode','25q','50q','75q'}
        for entry in asaf_dict:
          f.write(str(asaf_drugs[entry][row['drug_ID']]))
          f.write(' ')
        for el in asaf_drugs['5_high'][row['drug_ID']]:
          f.write(str(el))
          f.write(' ')
        for el in asaf_drugs['5_low'][row['drug_ID']]:
          f.write(str(el))
          f.write(' ')
        #gaaf
        gaaf_dict ={'min','max','upper_q','median','lower_q','mean','mode'}
        for entry in gaaf_dict:
          f.write(str(gaaf_drugs[entry][row['drug_ID']]))
          f.write(' ')
        #aaaf
        for el in aaaf_drugs[int(row['drug_ID'])]:
          f.write(str(el))
          f.write(' ')

        #Targets
        #ssaf
        ssaf_dict = {'mean', 'median', '75q', '85q', '95q'}
        for entry in ssaf_dict:
          f.write(str(ssaf_targets[entry][row['target_ID']]))
          f.write(' ')
        #asaf
        asaf_dict ={'mean_1','mean_2','number','mode','25q','50q','75q'}
        for entry in asaf_dict:
          f.write(str(asaf_targets[entry][row['target_ID']]))
          f.write(' ')

        for el in asaf_targets['5_high'][row['target_ID']]:
          f.write(str(el))
          f.write(' ')
        for el in asaf_targets['5_low'][row['target_ID']]:
          f.write(str(el))
          f.write(' ')
        #gaaf
        gaaf_dict ={'min','max','upper_q','median','lower_q','mean','mode'}
        for entry in gaaf_dict:
          f.write(str(gaaf_targets[entry][row['target_ID']]))
          f.write(' ')
        #aaaf
        for el in aaaf_targets[int(row['target_ID'])]:
          f.write(str(el))
          f.write(' ')
        f.write('\n')
      

In [48]:
write_file_RAW('features/train_features_KIBA.txt', kiba_train, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

In [49]:
write_file_RAW('features/combined/train_fold1_KIBA.txt', kiba_fold1, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW('features/combined/train_fold2_KIBA.txt', kiba_fold2, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW('features/combined/train_fold3_KIBA.txt', kiba_fold3, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW('features/combined/train_fold4_KIBA.txt', kiba_fold4, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW('features/combined/train_fold5_KIBA.txt', kiba_fold5, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

In [51]:
write_file_RAW('features/test_features_KIBA.txt', kiba_test, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

#Write file with Drugs and Targets split

In [53]:
def write_file_RAW_drugs(outpath:str, dataframe, ssaf, asaf, gaaf, aaaf):
  with open(outpath, 'w') as f:
      for index, row in dataframe.iterrows():
        #DRUGS
        #ssaf
        ssaf_dict = {'mean', 'median', '75q', '85q', '95q'}
        for entry in ssaf_dict:
          f.write(str(ssaf[entry][row['drug_ID']]))
          f.write(' ')
        #asaf
        asaf_dict ={'mean_1','mean_2','number','mode','25q','50q','75q'}
        for entry in asaf_dict:
          f.write(str(asaf[entry][row['drug_ID']]))
          f.write(' ')
        for el in asaf['5_high'][row['drug_ID']]:
          f.write(str(el))
          f.write(' ')
        for el in asaf['5_low'][row['drug_ID']]:
          f.write(str(el))
          f.write(' ')
        #gaaf
        gaaf_dict ={'min','max','upper_q','median','lower_q','mean','mode'}
        for entry in gaaf_dict:
          f.write(str(gaaf[entry][row['drug_ID']]))
          f.write(' ')
        #aaaf
        for el in aaaf[int(row['drug_ID'])]:
          f.write(str(el))
          f.write(' ')
        f.write('\n')

def write_file_RAW_targets(outpath:str, dataframe, ssaf, asaf, gaaf, aaaf):
  with open(outpath, 'w') as f:
      for index, row in dataframe.iterrows():
        #DRUGS
        #ssaf
        ssaf_dict = {'mean', 'median', '75q', '85q', '95q'}
        for entry in ssaf_dict:
          f.write(str(ssaf[entry][row['target_ID']]))
          f.write(' ')
        #asaf
        asaf_dict ={'mean_1','mean_2','number','mode','25q','50q','75q'}
        for entry in asaf_dict:
          f.write(str(asaf[entry][row['target_ID']]))
          f.write(' ')
        for el in asaf['5_high'][row['target_ID']]:
          f.write(str(el))
          f.write(' ')
        for el in asaf['5_low'][row['target_ID']]:
          f.write(str(el))
          f.write(' ')
        #gaaf
        gaaf_dict ={'min','max','upper_q','median','lower_q','mean','mode'}
        for entry in gaaf_dict:
          f.write(str(gaaf[entry][row['target_ID']]))
          f.write(' ')
        #aaaf
        for el in aaaf[int(row['target_ID'])]:
          f.write(str(el))
          f.write(' ')
        f.write('\n')
      

In [54]:
write_file_RAW_drugs('features/drugs/train_fold1_KIBA.txt', kiba_fold1, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA)
write_file_RAW_drugs('features/drugs/train_fold2_KIBA.txt', kiba_fold2, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA)
write_file_RAW_drugs('features/drugs/train_fold3_KIBA.txt', kiba_fold3, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA)
write_file_RAW_drugs('features/drugs/train_fold4_KIBA.txt', kiba_fold4, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA)
write_file_RAW_drugs('features/drugs/train_fold5_KIBA.txt', kiba_fold5, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA)

In [55]:
write_file_RAW_targets('features/targets/train_fold1_KIBA.txt', kiba_fold1, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW_targets('features/targets/train_fold2_KIBA.txt', kiba_fold2, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW_targets('features/targets/train_fold3_KIBA.txt', kiba_fold3, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW_targets('features/targets/train_fold4_KIBA.txt', kiba_fold4, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)
write_file_RAW_targets('features/targets/train_fold5_KIBA.txt', kiba_fold5, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)

In [56]:
write_file_RAW_drugs('features/drugs/test_KIBA.txt', kiba_test, ssaf_drugs_KIBA, asaf_drugs_KIBA, gaaf_drugs_KIBA, aaaf_drugs_KIBA)
write_file_RAW_targets('features/targets/test_KIBA.txt', kiba_test, ssaf_targets_KIBA, asaf_targets_KIBA, gaaf_targets_KIBA, aaaf_targets_KIBA)